In [1]:
# install dependencies
%pip install -q \
  more_itertools \
  pandas \
  pycaret \
  'pycaret[mlops]' \
  python_dotenv \
  python_multipart \
  openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
# global parameters
DATA_DIR = '../datasets/swell/final'
TEST_DATA_NAME = 'test'
API_NAME = 'stress-classifier'
API_PORT = 8081

In [3]:
from IPython.display import display_html

def greeting():
  api_playground_url = f'http://localhost:{API_PORT}/docs'
  display_html(
    f'<b>See API Playground in <a href="{api_playground_url}">{api_playground_url}</a></b>',
    raw=True
  )

greeting()

See API Playground in http://localhost:8081/docs

In [4]:
# set up the environment
import os
os.environ['PYCARET_CUSTOM_LOGGING_LEVEL'] = 'CRITICAL'

import pandas as pd
pd.set_option('display.max_columns', 128)

In [5]:
# prepare the data
from pathlib import Path
from pycaret.datasets import get_data
from zipfile import ZipFile

DATA = {
  name: None
  for name in ['train', TEST_DATA_NAME]
}

for data_name in DATA.keys():
  data_path = Path(DATA_DIR).joinpath(data_name)
  # extract the compressed data files
  ZipFile(data_path.with_suffix('.zip'), 'r').extract(
    str(data_path.with_suffix('.csv')), '..'
  )
  print(f'Data file "{data_name}" has been extracted successfully')
  # load the data
  print(f'Loading data file "{data_name}"')
  DATA[data_name] = get_data(dataset=f'{data_path}')

Data file "train" has been extracted successfully
Loading data file "train"


,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,SD1,SD2,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,sampen,higuci,datasetId,condition
0,885.157845,853.763730,140.972741,15.554505,15.553371,9.063146,69.499952,11.133333,0.533333,11.001565,199.061782,-0.856554,0.335218,-0.000203,-0.000179,0.017080,0.007969,0.007969,2.143342,-0.856554,0.335218,2661.894136,72.203287,1009.249419,27.375666,98.485263,15.522603,0.421047,1.514737,3686.666157,65.018055,0.015380,2.139754,1.163485,2,no stress
1,939.425371,948.357865,81.317742,12.964439,12.964195,6.272369,64.363150,5.600000,0.000000,9.170129,114.634458,-0.408190,-0.155286,-0.000059,0.000611,0.013978,0.004769,0.004769,2.930855,-0.408190,-0.155286,2314.265450,76.975728,690.113275,22.954139,99.695397,2.108525,0.070133,0.304603,3006.487251,327.296635,0.003055,2.174499,1.084711,2,interruption
2,898.186047,907.006860,84.497236,16.305279,16.305274,5.182201,67.450066,13.066667,0.200000,11.533417,118.939253,0.351789,-0.656813,-0.000011,-0.000263,0.018539,0.008716,0.008716,2.127053,0.351789,-0.656813,1373.887112,51.152225,1298.222619,48.335104,98.950472,13.769729,0.512671,1.049528,2685.879461,94.280910,0.010607,2.135350,1.176315,2,interruption
3,881.757865,893.460030,90.370537,15.720468,15.720068,5.748591,68.809562,11.800000,0.133333,11.119476,127.318597,-0.504947,-0.386138,0.000112,0.000494,0.017761,0.008660,0.008660,2.050988,-0.504947,-0.386138,2410.357408,70.180308,1005.981659,29.290305,98.224706,18.181913,0.529387,1.775294,3434.520980,55.328701,0.018074,2.178341,1.179688,2,no stress
4,809.625331,811.184865,62.766242,19.213819,19.213657,3.266724,74.565728,20.200000,0.200000,13.590641,87.718281,-0.548408,-0.154252,-0.000100,-0.002736,0.023715,0.013055,0.013055,1.816544,-0.548408,-0.154252,1151.177330,43.918366,1421.782051,54.242160,96.720007,48.215822,1.839473,3.279993,2621.175204,29.487873,0.033912,2.221121,1.249612,2,no stress


Data file "test" has been extracted successfully
Loading data file "test"


,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,SD1,SD2,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,sampen,higuci,datasetId,condition
0,721.901897,727.267280,74.722315,12.361264,12.361069,6.044877,84.121868,4.933333,0.000000,8.743513,105.310967,1.262958,-0.703779,0.000081,-0.000951,0.017605,0.011208,0.011208,1.570800,1.262958,-0.703779,1016.073759,59.818117,615.914573,36.260015,90.239711,66.617057,3.921868,9.760289,1698.605390,9.245599,0.108160,2.097342,1.243696,2,no stress
1,843.538633,844.407930,58.499429,19.298880,19.298795,3.031234,71.478642,21.000000,0.200000,13.650863,81.596693,-0.445806,-0.144911,0.000061,-0.001543,0.022969,0.011970,0.011970,1.918953,-0.445806,-0.144911,765.518473,32.452560,1566.866135,66.424024,98.336849,26.500086,1.123416,1.663151,2358.884694,59.126832,0.016913,2.217275,1.250056,2,time pressure
2,958.523868,966.671125,132.849110,21.342715,21.342653,6.224565,63.874293,24.133333,1.800000,15.096571,187.269500,-0.666523,-0.103725,-0.000055,-0.002000,0.022079,0.009606,0.009606,2.298473,-0.666523,-0.103725,2237.739905,51.696218,2074.868884,47.933575,99.233584,16.024935,0.370208,0.766416,4328.633724,129.477524,0.007723,2.217136,1.144943,2,no stress
3,824.838669,842.485905,117.822094,11.771814,11.771248,10.008830,74.330531,4.733333,0.533333,8.326307,166.417439,-0.698096,-0.145092,0.000130,0.000445,0.014920,0.007051,0.007051,2.116033,-0.698096,-0.145092,2330.980957,81.661325,505.886664,17.722743,96.641348,17.581470,0.615932,3.358652,2854.449091,28.773854,0.034754,2.106863,1.142355,2,no stress
4,756.707933,747.941620,143.968457,13.357748,13.356388,10.777899,82.092049,5.933333,0.666667,9.447545,203.382835,1.134111,0.769517,0.000310,-0.000170,0.019649,0.011689,0.011689,1.681034,1.134111,0.769517,4750.624447,89.465158,524.203971,9.871963,93.707747,35.199054,0.662879,6.292253,5310.027472,14.892559,0.067148,1.912191,1.128098,2,interruption


In [6]:
# load the experiment and the model
from pathlib import Path
from pycaret.classification import load_experiment

model_dir = Path(f'../models/{TEST_DATA_NAME}')

exp = load_experiment(
  path_or_file=model_dir.joinpath('experiment.pkl'),
  data=DATA['train'],
  test_data=DATA[TEST_DATA_NAME],
)
from IPython.display import display_html
display_html(exp.dataset_transformed.head(5))

# load the model
model = exp.load_model(model_name=model_dir.joinpath('model'))
display_html(model)

,Description,Value
0,Session id,123
1,Target,condition
2,Target type,Multiclass
3,Target mapping,"interruption: 0, no stress: 1, time pressure: 2"
4,Original data shape,"(410322, 36)"
5,Transformed data shape,"(410322, 29)"
6,Transformed train set shape,"(369289, 29)"
7,Transformed test set shape,"(41033, 29)"
8,Ignore features,1
9,Numeric features,34


,MEAN_RR,MEDIAN_RR,RMSSD,SDRR_RMSSD,HR,pNN25,pNN50,SD2,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,sampen,higuci,condition
0,885.157837,853.763733,15.554504,9.063146,69.499954,11.133333,0.533333,199.061783,-0.856554,0.335218,-0.000203,-0.000179,0.017080,0.007969,2.143342,2661.894043,72.203285,1009.249390,27.375666,98.485260,15.522602,0.421047,1.514737,3686.666260,65.018051,0.015380,2.139754,1.163485,1
1,939.425354,948.357849,12.964439,6.272368,64.363152,5.600000,0.000000,114.634460,-0.408190,-0.155286,-0.000059,0.000611,0.013978,0.004769,2.930855,2314.265381,76.975731,690.113281,22.954140,99.695396,2.108526,0.070133,0.304603,3006.487305,327.296631,0.003055,2.174499,1.084711,0
2,898.186035,907.006836,16.305279,5.182201,67.450066,13.066667,0.200000,118.939255,0.351789,-0.656813,-0.000011,-0.000263,0.018539,0.008716,2.127053,1373.887085,51.152225,1298.222656,48.335102,98.950470,13.769730,0.512671,1.049528,2685.879395,94.280907,0.010607,2.135350,1.176315,0
3,881.757874,893.460022,15.720469,5.748590,68.809563,11.800000,0.133333,127.318596,-0.504947,-0.386138,0.000112,0.000494,0.017761,0.008660,2.050988,2410.357422,70.180305,1005.981689,29.290304,98.224709,18.181913,0.529387,1.775294,3434.520996,55.328701,0.018074,2.178341,1.179688,1
4,809.625305,811.184875,19.213820,3.266724,74.565727,20.200001,0.200000,87.718277,-0.548408,-0.154252,-0.000100,-0.002736,0.023715,0.013055,1.816544,1151.177368,43.918365,1421.782104,54.242161,96.720009,48.215824,1.839473,3.279993,2621.175293,29.487873,0.033912,2.221121,1.249612,1


Transformation Pipeline and Model Successfully Loaded


Pipeline(memory=FastMemory(location=/var/folders/v9/1fbzf6vj1cs0xqpnkjny3mth0000gn/T/joblib),
 steps=[('label_encoding',
 TransformerWrapperWithInverse(exclude=None, include=None,
 transformer=LabelEncoder())),
 ('remove_multicollinearity',
 TransformerWrapper(exclude=[], include=None,
 transformer=RemoveMulticollinearity(threshold=0.999))),
 ('trained_model',
 XGBC...
 importance_type=None,
 interaction_constraints=None, learning_rate=None,
 max_bin=None, max_cat_threshold=None,
 max_cat_to_onehot=None, max_delta_step=None,
 max_depth=None, max_leaves=None,
 min_child_weight=None, missing=nan,
 monotone_constraints=None, multi_strategy=None,
 n_estimators=None, n_jobs=-1,
 num_parallel_tree=None,
 objective='multi:softprob', ...))],
 verbose=False) In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook. On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org. Pipeline Pipeline(memory=FastMemory(location=/var/folders/v9/1fbzf6vj1cs0xqpnkjny3mth0000gn/T/joblib),
 steps=[('label_encoding',
 TransformerWrapperWithInverse(exclude=None, include=None,
 transformer=LabelEncoder())),
 ('remove_multicollinearity',
 TransformerWrapper(exclude=[], include=None,
 transformer=RemoveMulticollinearity(threshold=0.999))),
 ('trained_model',
 XGBC...
 importance_type=None,
 interaction_constraints=None, learning_rate=None,
 max_bin=None, max_cat_threshold=None,
 max_cat_to_onehot=None, max_delta_step=None,
 max_depth=None, max_leaves=None,
 min_child_weight=None, missing=nan,
 monotone_constraints=None, multi_strategy=None,
 n_estimators=None, n_jobs=-1,
 num_parallel_tree=None,
 objective='multi:softprob', ...))],
 verbose=False) label_encoding: TransformerWrapperWithInverse TransformerWrapperWithInverse(exclude=None, include=None,
 transformer=LabelEncoder()) transformer: LabelEncoder LabelEncoder() LabelEncoder LabelEncoder() remove_multicollinearity: TransformerWrapper TransformerWrapper(exclude=[], include=None,
 transformer=RemoveMulticollinearity(threshold=0.999)) transformer: RemoveMulticollinearity RemoveMulticollinearity(threshold=0.999) RemoveMulticollinearity RemoveMulticollinearity(threshold=0.999) XGBClassifier XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
 colsample_bylevel=None, colsample_bynode=None,
 colsample_bytree=None, device='cpu', early_stopping_rounds=None,
 enable_categorical=False, eval_metric=None, feature_types=None,
 gamma=None, grow_policy=None, importance_type=None,
 interaction_constraints=None, learning_rate=None, max_bin=None,
 max_cat_threshold=None, max_cat_to_onehot=None,
 max_delta_step=None, max_depth=None, max_leaves=None,
 min_child_weight=None, missing=nan, monotone_constraints=None,
 multi_strategy=None, n_estimators=None, n_jobs=-1,
 num_parallel_tree=None, objective='multi:softprob', ...)

In [7]:
# from openai import OpenAI
# from dotenv import load_dotenv

# load_dotenv()
# client = OpenAI()

# response = client.chat.completions.create(
#   model='gpt-3.5-turbo',
#   messages=[
#     {
#       'role': 'system',
#       'content': 'Be a doctor and help user with the stress',
#     },
#     {
#       'role': 'user',
#       'content': 'I am stressed out',
#     },
#     {
#       'role': 'assistant',
#       'content': 'I recommend you to do some meditation',
#     },
#     {
#       'role': 'user',
#       'content': '我壓力好大',
#     }
#   ],
#   stream=True,
# )

# for s in response:
#   s = s.choices[0].delta.content
#   if not s:
#     continue
#   print(s, end='', flush=True)

# 了解，壓力很大的時候可以嘗試以下幾種方法來幫助自己紓壓：

# 1. 深呼吸：專注地深呼吸可以幫助你放鬆身心。慢慢吸氣，然後再慢慢吐氣，並專注於呼吸的感覺，將注意力從壓力上轉移開來。

# 2. 運動：運動釋放出內啡肽，這是一種能夠讓你感到快樂和放鬆的天然化學物質。試試散步、跑步、瑜伽等活動，讓身體活動起來，減輕壓力。

# 3. 與他人交流：和朋友、家人或專業人士分享你的感受和困擾。有人傾聽和理解你的感受可以減輕壓力，並可能提供有用的建議和支持。

# 4. 時間管理：好的時間管理可以幫助你更有效地處理壓力。確定優先事項，制定一個計劃，合理分配時間，並為自己留出一些休息和放鬆的時間。

# 5. 飲食和睡眠：保持均衡的飲食和良好的睡眠習慣對於減輕壓力非常重要。嘗試避免過多的咖啡因和糖分，並確保每晚獲得充足的睡眠。

# 6. 創造性活動：找一個可以讓你放鬆和發洩的創造性活動，例如繪畫、寫作、唱歌等。這些活動可以轉移注意力，幫助你釋放壓力。

# 如果你覺得壓力無法自行緩解，或者壓力對你的日常生活造成了嚴重的困擾，建議尋求專業的心理輔導或醫療支持。

In [8]:
# implement API
from fastapi import FastAPI, HTTPException
from fastapi.responses import StreamingResponse, RedirectResponse
from more_itertools import ichunked
import numpy as np
import pandas as pd
from pydantic import BaseModel
import utils.hrv_feature_extraction as hfe

# define constants
SAMPLE_WINDOW_SIZE = 400
STRESS_LEVELS = {
  'no stress': b'0',
  'interruption': b'1',
  'time pressure': b'2',
}

# define schemas
ERR_INVALID_CONTENT_TYPE = HTTPException(
  status_code=400,
  detail='Invald content type',
)

ERR_VALUE_OUT_OF_RANGE = HTTPException(
  status_code=400,
  detail='Value is out of range',
)

class LevelSeriesRequest(BaseModel):
  rr_intervals: list[float] = np.random.randint(low=700, high=900, size=100).tolist()

# define features
api = FastAPI()

@api.post(path=f'/{API_NAME}/level-series')
async def level_series(args: LevelSeriesRequest) -> StreamingResponse:
  rr_intervals = np.array(args.rr_intervals)

  if len(rr_intervals) <= 10:
    raise ERR_VALUE_OUT_OF_RANGE

  def _iter_levels():
    for features in ichunked(_iter_features(), 1024):
      for label in exp.predict_model(
        estimator=model,
        data=pd.DataFrame(features),
        verbose=False,
      )['prediction_label']:
        yield STRESS_LEVELS[label]

  def _iter_features():
    for window in hfe.get_window_iterator(
      values=rr_intervals,
      window_size=SAMPLE_WINDOW_SIZE,
    ):
      yield hfe.extract_hrv_features_from_rri_window(
        rri_window=window,
      )

  return StreamingResponse(
    content=_iter_levels(),
    headers={
      'X-Level-Series-Size': str(1 + max(0, len(rr_intervals) - SAMPLE_WINDOW_SIZE)),
    },
  )

@api.get(path='/')
async def entry():
  return RedirectResponse(url='/docs', status_code=301)

In [10]:
# run the API services
from uvicorn import Config, Server
from os import cpu_count

# start the serving loop
greeting()
await Server(Config(
  app=api,
  host='localhost',
  port=API_PORT,
  loop='asyncio',
  access_log=False,
  use_colors=True,
  workers=cpu_count(),
)).serve()

See API Playground in http://localhost:8081/docs

INFO:     Started server process [87015]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8081 (Press CTRL+C to quit)


In [ ]:
exp.models()

In [ ]:
# clean up
exit(0)

: 